#### Assumptions
* Prijzen worden ingeschat op een bepaald jaar, vanaf dat jaar worden prijzen geescaleerd naar de toekomst. 
* price_reference_year (dan NPV 1 en de escalation is 1)
* price_reference_year + 1 CAPEX start
* CAPEX are distributed via the allocation factors, allocation starts immediately in startyear
* CAPEX is escalated with escalation factors
* escalation values start on the start year (if startyear is 2023, then the CAPEX values are 1.02 * Capex values)
* OPEX values are the sum of the escalated CAPEX values, and OPEX values are themselves not escalated
* all things have decommissioning costs
* some things may have residual value
* automatisch herinvesteren

#### Potential to do's
* add geographical data to the system objects (to plot them in space, but also to possibly calculate distances)
* add 'owner' per system object (is the business case for each stakeholder positive?)
* now in case of an error, a default value is returned. The exeptions should return a clear error statement
* develop tests
* check what are now input fields and what is read from the xls (make sure this is handeled properly

#### 0. Import basic methods from source.py
Import useful methods from source.py (NB: can later be integrated into a package)

In [1]:
from source import *

#### 1. Open xls file and ingest Inputs table
Note I am assuming that all inputs will be aggregated into one single inputs table

In [2]:
filename = r'H2 Model - Input sheet.xlsm'

In [3]:
%%time
# load workbook
wb = load_workbook(filename)

CPU times: total: 375 ms
Wall time: 498 ms


In [4]:
# load worksheet
ws = wb["Input Tab"]

In [5]:
mapping = {}

for entry, data_boundary in ws.tables.items():
    # parse the data within the ref boundary
    data = ws[data_boundary]
    
    # extract the data (the inner list comprehension gets the values for each cell in the table)
    content = [[cell.value for cell in ent] for ent in data]
    
    # find header
    header = content[0]
    
    # find the rest ... the contents, excluding the header
    rest = content[1:]
    
    # create dataframe with the column names
    # and pair table name with dataframe
    df = pd.DataFrame(rest, columns = header)
    mapping[entry] = df

In [6]:
Inputs = pd.DataFrame(mapping['Scenario1'])
Inputs.head()

,Category,Sub-system,Element,Component,Description,Number,Unit,Comments
0,System input,Wind energy source & Transport,Offshore wind park,-,Total Windpark Capacity,3000,MW,None
1,System input,Wind energy source & Transport,Offshore wind park,Turbine,Windturbine Capacity per unit,20,MW,None
2,System input,Wind energy source & Transport,Offshore wind park,Inter-array cable,Length,50,km,None
3,System input,AC Substation & Transport,AC Substation & Transport,AC Substation,Capacity per unit,2000,MW,None
4,System input,AC Substation & Transport,AC Substation & Transport,AC collection cable,Length,25,km,None


#### 2. Data inspection 

In [7]:
Inputs.columns

Index(['Category', 'Sub-system', 'Element', 'Component', 'Description',
       'Number', 'Unit', 'Comments'],
      dtype='object')

In [8]:
Inputs[Inputs['Sub-system']=='Wind energy source & Transport'].head()

,Category,Sub-system,Element,Component,Description,Number,Unit,Comments
0,System input,Wind energy source & Transport,Offshore wind park,-,Total Windpark Capacity,3000,MW,None
1,System input,Wind energy source & Transport,Offshore wind park,Turbine,Windturbine Capacity per unit,20,MW,None
2,System input,Wind energy source & Transport,Offshore wind park,Inter-array cable,Length,50,km,None
14,System input,Wind energy source & Transport,Offshore wind park,Inter-array cable,Transport loss,0.02,%,None
23,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Foundation & cable costs (Development and Proj...,207000,EUR / MW,"0,1 * Foundation procurement"


In [9]:
list(Inputs['Sub-system'].unique())

['Wind energy source & Transport',
 'AC Substation & Transport',
 'Converter & Transport',
 'Electrolysis, Electricity & Transport',
 '-']

In [10]:
list(Inputs['Element'].unique())

['Offshore wind park',
 'AC Substation & Transport',
 'Converter',
 'Electrolysis',
 'Electricity',
 'Facility location',
 'Transport',
 '-']

In [11]:
list(Inputs['Component'].unique())

['-',
 'Turbine',
 'Inter-array cable',
 'AC Substation',
 'AC collection cable',
 'HVDC Converter',
 'HVDC cable',
 'Electrolyser',
 'DCAC Converter',
 'Artificial island',
 'Mainland',
 'H2 Pipeline',
 'Electricity cable',
 'HVDC Cable',
 'Foundation & cable',
 'DC collection cable',
 'Desalination unit',
 'Compressor unit',
 'Storage unit',
 'Compressor after storage']

In [12]:
subsystem='Wind energy source & Transport'
element='Offshore wind park'
component='Foundation & cable'

Inputs[
    (Inputs['Sub-system'] == subsystem) &
    (Inputs['Element'] == element) &
    (Inputs['Component'] == component) 
]

,Category,Sub-system,Element,Component,Description,Number,Unit,Comments
23,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Foundation & cable costs (Development and Proj...,207000,EUR / MW,"0,1 * Foundation procurement"
24,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Foundation & cable costs (Procurement),2070000,EUR / MW,"1,8 * Turbine procurement"
25,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Foundation & cable costs (Installation and Com...,414000,EUR / MW,"0,2 * Foundation procurement"
26,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Number of units,3000,MW,Unit = MW
27,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Construction duration,3,years,None
28,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Share of Investments in Year 1,0.4,% of Capex total,None
29,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Share of Investments in Year 2,0.3,% of Capex total,None
30,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Share of Investments in Year 3,0.3,% of Capex total,None
31,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Economic Lifetime,50,years,None
32,Capex,Wind energy source & Transport,Offshore wind park,Foundation & cable,Depreciation Flag,1,flag,None


#### 3. Select one item from the Inputs table (generate cashflow overview)

##### A. do per item

In [13]:
%%time
df_Foundations = Inputs_2_cashflow(Inputs, 
                                   startyear=2023,
                                   lifecycle=50,
                                   subsystem='Wind energy source & Transport',
                                   element='Offshore wind park',
                                   component='Foundation & cable',
                                   Debug=False)

CPU times: total: 62.5 ms
Wall time: 57.4 ms


In [14]:
%%time
df_npv_Foundations = calculate_npv(df_Foundations,
                                baseyear=Inputs[Inputs.Description == 'Escalation base year'].Number.item(),
                                WACC=Inputs[Inputs.Description == 'WACC'].Number.item())

CPU times: total: 15.6 ms
Wall time: 15 ms


In [ ]:
%%time
lims = [-15000, 15000]
create_npv_plot(df_npv_Foundations, cash_flow_lims=lims, npv_lims=lims)

In [ ]:
%%time
df_Turbines = Inputs_2_cashflow(Inputs, 
                                   startyear=2023,
                                   lifecycle=50,
                                   subsystem='Wind energy source & Transport',
                                   element='Offshore wind park',
                                   component='Turbine',
                                   Debug=False)

In [ ]:
%%time
df_npv_Turbines = calculate_npv(df_Turbines,
                                baseyear=Inputs[Inputs.Description == 'Escalation base year'].Number.item(),
                                WACC=Inputs[Inputs.Description == 'WACC'].Number.item())

In [ ]:
%%time
lims = [-15000, 15000]
create_npv_plot(df_npv_Turbines, cash_flow_lims=lims, npv_lims=lims)

##### B. combine two items

In [ ]:
df_Foundations = Inputs_2_cashflow(Inputs, 
                                   startyear=2023,
                                   lifecycle=50,
                                   subsystem='Wind energy source & Transport',
                                   element='Offshore wind park',
                                   component='Foundation & cable',
                                   Debug=False)
df_Turbines = Inputs_2_cashflow(Inputs,
                                startyear=2023,
                                lifecycle=50,
                                subsystem='Wind energy source & Transport',
                                element='Offshore wind park',
                                component='Turbine',
                                Debug=False)
df_combined = combine_cashflow_dataframes([df_Foundations, df_Turbines])

In [ ]:
df_Foundations

In [ ]:
df_Turbines

In [ ]:
df_npv_combined = calculate_npv(df_combined)
df_npv_combined

In [ ]:
%%time
lims = [-15000, 15000]
create_npv_plot(df_npv_combined, cash_flow_lims=lims, npv_lims=lims)